In [38]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Dropout


from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders
print(tf.__version__)

2.18.0


In [39]:
img_width = 128
img_height = 128

In [40]:
splitfolders.ratio("brain_tumor_dataset", output="output", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default valuesode\Projects\BrainTumorDetection_CNN\

Copying files: 253 files [00:00, 285.07 files/s]


In [41]:
train_datagen=ImageDataGenerator(rescale=1/255.0)
test_datagen=ImageDataGenerator(rescale=1/255.0)

In [42]:
train_data_generator=train_datagen.flow_from_directory(directory='output/train',
                                                       target_size=(img_height,img_width),
                                                       class_mode='categorical',
                                                       batch_size=16)

Found 202 images belonging to 2 classes.


In [43]:
validation_data_generator=test_datagen.flow_from_directory(directory='output/test',
                                                     target_size=(img_height,img_width),
                                                     class_mode='categorical',
                                                     batch_size=16)

Found 27 images belonging to 2 classes.


In [44]:
test_data_generator=test_datagen.flow_from_directory(directory='output/val',
                                                        target_size=(img_height,img_width),
                                                        class_mode='categorical',
                                                        batch_size=16)

Found 51 images belonging to 2 classes.


In [45]:

from tensorflow.keras import Input

model = Sequential([
    Input(shape=(img_height, img_width, 3)),  # Input layer
    Conv2D(16, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Dropout(0.2),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Dropout(0.3),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])


In [46]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
from tensorflow.keras.applications import VGG16 #Densenet Resnet efficientnet inception
#xception
#transformers---attention based networks 
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
def prepare_model():
    # Load VGG16 with pre-trained weights from ImageNet and exclude the top layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
    # Freeze the base model to retain pre-trained features during the initial training phase
    base_model.trainable = False
    
    # Add custom layers on top of the base VGG16 model
    model = Sequential()
    model.add(base_model)  # Add the VGG16 convolutional base
    
    # Add a global average pooling layer
    model.add(GlobalAveragePooling2D())
    
    # Add a fully connected layer
    model.add(Dense(16, activation='relu'))
    
    # Add the final output layer for multi-class classification (4 classes)
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [48]:
model.fit(train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 10,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 381ms/step - accuracy: 0.5708 - loss: 1.0879 - val_accuracy: 0.6667 - val_loss: 0.5780
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.8123 - loss: 0.4775 - val_accuracy: 0.8148 - val_loss: 0.5758
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 206ms/step - accuracy: 0.7773 - loss: 0.5283 - val_accuracy: 0.7407 - val_loss: 0.5444
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 138ms/step - accuracy: 0.7988 - loss: 0.4847 - val_accuracy: 0.8148 - val_loss: 0.5079
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 219ms/step - accuracy: 0.8073 - loss: 0.3998 - val_accuracy: 0.8148 - val_loss: 0.4820
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.8304 - loss: 0.3505 - val_accuracy: 0.8519 - val_loss: 0.4694
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 0.8461 - loss: 0.3250 - val_accuracy: 0.8519 - val_loss: 0.4728
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.8693 - loss: 0.3075 - val_accuracy: 0.

In [49]:
# Evaluate the Model
score = model.evaluate(test_data_generator, steps=len(test_data_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.8294 - loss: 0.4758 
Test loss: 0.4659384489059448
Test accuracy: 0.8235294222831726


In [50]:
import numpy as np
from keras.preprocessing import image
from PIL import Image

In [51]:

# Prepare the image for prediction
import os
img_path1 = os.path.join('output', 'val', 'yes', 'Y2.jpg')  # Adjusted for relative path
img_yes= image.load_img(img_path1, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_yes)
img_arr = np.expand_dims(img_arr, axis=0)
result=model.predict(img_arr)
result


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step


array([[0., 1.]], dtype=float32)

In [52]:
# Prepare the image for prediction
import os
img_path2 = os.path.join('output', 'val', 'no', '3 no.jpg')  # Adjusted for relative path
img_no = image.load_img(img_path2, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_no)
img_arr = np.expand_dims(img_arr, axis=0)
result=model.predict(img_arr)
result


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


array([[1., 0.]], dtype=float32)

In [53]:
model_json=model.to_json() # is a format popular in saving data like csv or xml style
json_file=open("Brain_tumor_model.json","w")
json_file.write(model_json)

model.save_weights("model.weights.h5") # to save in keras native  format 
print("Saved model to disk")
  

Saved model to disk


In [54]:
from tensorflow.keras.models import model_from_json
json_file=open('Brain_tumor_model.json','r')
loaded_model_json=json_file.read()
json_file.close()

loaded_model=model_from_json(loaded_model_json) # load the saved model 
loaded_model.load_weights("model.weights.h5") 
print("Loaded model from disk")



Loaded model from disk


In [55]:
loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
score=model.evaluate(validation_data_generator, steps=len(validation_data_generator))

# Print the evaluation score
print('Test loss:', score[0])   
print('Test accuracy:', score[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.7932 - loss: 0.6570
Test loss: 0.5658267140388489
Test accuracy: 0.8148148059844971


In [56]:
loaded_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 28800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │     1,843,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,848,482 (7.05 MB)

 Trainable params: 1,848,482 (7.05 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:

# Prepare the image for prediction
import os
img_path1 = os.path.join('output', 'val', 'yes', 'Y2.jpg')  # Adjusted for relative path
img_yes= image.load_img(img_path1, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_yes)
img_arr = np.expand_dims(img_arr, axis=0)

result=model.predict(img_arr)
result


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0., 1.]], dtype=float32)

In [58]:

# Decode the prediction to a meaningful class label
classes = {0: 'No', 1: 'Yes'}  # Example class mapping
predicted_class = np.argmax(result)
print(f"Predicted Class: {classes[predicted_class]} ({result})")


Predicted Class: Yes ([[0. 1.]])
